In [1]:
import os
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt

In [4]:
folder_path = os.path.join('dataset','processed')

dfs = []

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)

        dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

In [5]:
combined_df.head(2)

,Unnamed: 0,time,Latitude,Longitude,signature_label,icao24,trip_number
0,0,1645630911,28.210749,-81.275455,0,a007c6,2
1,1,1645630912,28.210656,-81.275778,0,a007c6,2


## Train-Test

In [30]:
# 0d02a8 = 0, 2 - BAD; a007c6 = 0, 4
flights_ids_to_include = ['0d02a8', 'a007c6']
flights_trips_to_include_tr = [0, 1]
flights_trips_to_include_test = [2, 3, 4]


train_data = combined_df[(combined_df['icao24'].isin(flights_ids_to_include)) & (combined_df['trip_number'].isin(flights_trips_to_include_tr))]
test_data = combined_df[(combined_df['icao24'].isin(flights_ids_to_include)) & (combined_df['trip_number'].isin(flights_trips_to_include_test))]

In [31]:
train_data.head()

,Unnamed: 0,time,Latitude,Longitude,signature_label,icao24,trip_number
2128,0,1645565574,27.921707,-81.968839,0,a007c6,1
2129,1,1645565575,27.921707,-81.968839,0,a007c6,1
2130,2,1645565576,27.921707,-81.968839,0,a007c6,1
2131,3,1645565577,27.921707,-81.968839,0,a007c6,1
2132,4,1645565578,27.921707,-81.968839,0,a007c6,1


In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

columns_to_drop = ['signature_label', 'icao24', 'trip_number', 'Unnamed: 0']

y_train = train_data['signature_label']
X_train = train_data.drop(columns_to_drop, axis=1)

y_test = test_data['signature_label']
X_test = test_data.drop(columns_to_drop, axis=1)

X_train.head(), y_test.head()

(            time   Latitude  Longitude
 2128  1645565574  27.921707 -81.968839
 2129  1645565575  27.921707 -81.968839
 2130  1645565576  27.921707 -81.968839
 2131  1645565577  27.921707 -81.968839
 2132  1645565578  27.921707 -81.968839,
 0    0
 1    0
 2    0
 3    0
 4    0
 Name: signature_label, dtype: int64)

In [33]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)


clf = RandomForestClassifier()
clf.fit(X_train_scaled, y_train)

y_pred = clf.predict(X_test_scaled)

In [34]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.998006379585327
